In [14]:
# University of York
# Module - Advanced programming
# Summative Assessment 

In [15]:
import json
from matplotlib import pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import pandas as pd
from pathlib import Path
import seaborn as sns
import tkinter
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from tkinter.ttk import *
from tkinter.filedialog import askopenfilename, asksaveasfile


In [16]:
# Create main function of the app

In [17]:
def main():
    
    # Create root window
    rootWindow = tkinter.Tk()
    rootWindow.title("STORE AUDIT")
    rootWindow.geometry("800x650")
    
    # Variable to hold the Global DataFrame to be used for processing visualizations 
    # Inspection score per year - type vendor seating and zip codes
    df = pd.DataFrame()
    # Variable to hold the DataFrame to be for processing visualizations
    # Number of establishments per each type of violations
    # Stores DataFrame from the cleaned violation.json file
    dfViolations = pd.DataFrame()
    
    # Global variable used for retrieving the Entry text input 
    global rangeValues 
    rangeValues = StringVar()
    # Global variable to be used for output and visualization 
    # Default integer value set to 5
    # In case user clicks graph buttons with no initial data entry
    global rangeValuesInteger
    rangeValuesInteger = 5
    # Methods for opening/loading, saving and closing files required by app ( xlsx and json)
    
    # Check what happens when user wants to close main window from
    # X button or ALT+F4
    # 
    def on_closing():
        if messagebox.askokcancel('Quit', 'Do you want to quit?\nMake sure you have saved your work.'):
            rootWindow.destroy()
    # Mouse clicking
    
    # Open initial data ( csv)
    # Method that allows user to open a xlsx file
    # Returns  panda DataFrame
    def openInitialData():
        
        try:
            global df
            global fileNamePath
            # Open window and allows to chose CSV files
            # Issues with askopenfile
            # The returned file object contains at the beginning 
            # of the new file ï»¿ byte order mark chars
            # Could fix issue by getting the path of file and pass it to Pandas read_csv()
            fileName = askopenfilename(initialdir = "/", title = "Select initial file",
                                                filetypes = (('CSV Files', "*.csv"), ))
            # If user presses cancel, the returned object is None
            if not fileName:
                return None
            # Convert fileName to Path object to access the name attribute
            # Print the fileName.name into output area   
            fileNamePath = Path(fileName)
            fileNameTitle = fileNamePath.name
            print(fileNameTitle)
            # Read the CSV and convert it to panda DataFrame
            df = pd.read_csv(fileName)
            initialDimension = df.shape
            # CLEAN INITIAL CSV DATA
        
            # Filling in any missing data values with 0
            df.fillna(0, inplace=True)
        
            # Remove duplicates, if any, and display in console to check
            df.drop_duplicates(inplace=True)
            dimensionDfAfterDropDuplicates = df.shape
            print(df.shape)
        
            # Client request
            # Remove rows that have "PROGRAM STATUS" == Inactive
            # Check that column "PROGRAM STATUS" exists in the loaded initial DF
            if "PROGRAM STATUS" in df.columns:
                
                # Create filter for Inactive status
                filterProgramStatusInactive = (df["PROGRAM STATUS"] == "INACTIVE")
                # Remove(delete) rows that have the Program Status Inactive
                # Assigned to df to apply changes
                
                df = df.drop(index=df[filterProgramStatusInactive].index)
                
            else:
                pass
            # Display message to user regarding Program Status
            # Only if column Program status is available
            if "PROGRAM STATUS" in df.columns:
                
                dfShapeInactive = df.shape
                # Display message to user with loading and cleaning process
                message = """File {} was loaded.\n
Initial dimension of rows and columns {}\n
Dimension after duplicates were removed {}\n
Dimension after removing Program status - inactive {}""".format(fileNameTitle, initialDimension, dimensionDfAfterDropDuplicates, dfShapeInactive)
                output.delete(0.0, tkinter.END)
                output.insert(tkinter.END, message)
            else:
                message = """File {} was loaded.\n
Initial dimension of rows and columns {}\n
Dimension after duplicates were removed {}
                         """.format(fileNameTitle, initialDimension, dimensionDfAfterDropDuplicates)
                output.delete(0.0, tkinter.END)
                output.insert(tkinter.END, message)
            
        
        except FileNotFoundError:
            message = "Not able to load file.\nTry reloading it again."
            output.delete(0.0, tkinter.END)
            output.insert(tkinter.END, message)
        except KeyboardInterrupt:
            df.to_csv("Checkpoint_" + fileNameTitle)
            crashedWindow = tkinter.Toplevel()
            crashedWindow.configure(background='#084B8A')
            infoMessage = tkinter.Label(crashedWindow, text = """Program was closed using CTRL+C or Delete key.
            A checkpoint file has been saved in the main directory.""", font = ('Times New Roman', 16))
            infoMessage.pack()
        except GeneratorExit:
            df.to_csv("Checkpoint_" + fileNameTitle)
            crashedWindow = tkinter.Toplevel()
            crashedWindow.configure(background='#084B8A')
            infoMessage = tkinter.Label(crashedWindow, text = """The program closed unexpectedly.
            A checkpoint file has been saved in the main directory.""", font = ('Times New Roman', 16))
            infoMessage.pack()
        except RuntimeError:
            df.to_csv("Checkpoint_" + fileNameTitle)
            crashedWindow = tkinter.Toplevel()
            crashedWindow.configure(background='#084B8A')
            infoMessage = tkinter.Label(crashedWindow, text = """The program closed unexpectedly.
            A checkpoint file has been saved in the main directory.""", font = ('Times New Roman', 16))
            infoMessage.pack()
            
            
    # Callback method to save the cleaned DataFrame to Json format
    def saveCleanedData():
        global df
        global fileNamePath
        
        
        try:
            # Open file browser
            saveFileLocation = asksaveasfile(mode='w', initialdir="/", title="Save file",
            defaultextension=".json", filetypes=(('Json file', '*.json'),))
            # If user doesn't press cancel button
            if saveFileLocation != None:
                df.to_json(saveFileLocation)
                message = "File saved successfully"
                output.delete(0.0, tkinter.END)
                output.insert(tkinter.END, message)
            else:
                return None
        except FileNotFoundError:
            message = "Not able to save file.\nTry again."
            output.delete(0.0, tkinter.END)
            output.insert(tkinter.END, message)
        except ValueError:
            message = "Not able to encode json file."
            output.delete(0.0, tkinter.END)
            output.insert(tkinter.END, message)
            
            
    # Callback method to load the json cleaned DataFrame    
    def loadCleanedData():
        global df
        global dfViolations
        try:
            
            # Browse and open the json file
            fileName = askopenfilename(initialdir = "/", title = "Select cleaned file",
                                                filetypes = (('Json Files', "*.json"), ))
            
            # If user presses cancel, the returned object is None
            if not fileName:
                return None
            # Convert fileName to Path object to access the name attribute
            # Print the fileName.name into output area 
            fileNamePath = Path(fileName)
            fileNameTitle = fileNamePath.name
            
            # Read the new json file and convert to Pandas DataFrame
            newDataFrame = pd.read_json(fileName)
            # If column violation code is present
            # Assign the DataFrame to global dfViolations
            
            if 'VIOLATION CODE' in newDataFrame:
                print('Violation code column is present')
                dfViolations = newDataFrame
                print('dfViolations loaded')
            # Else assign it to global df
            else:
                df = newDataFrame
                print('dfGlobal loaded')
            
            # Feedback for user
            message = """File {} was loaded.""".format(fileNameTitle)
            output.delete(0.0, tkinter.END)
            output.insert(tkinter.END, message)
        except FileNotFoundError:
            message = "Not able to load file.\nTry reloading it again."
            output.delete(0.0, tkinter.END)
            output.insert(tkinter.END, message)
     
    # Callback methods for the client's requirements
    # Buttons displayed in the main window
    
    # Method to retrieve the input entered by the user from the entry box
    def getEntryValue():
        global rangeValues
        global rangeValuesInteger
        
        try:
            
            # Get the StringVal from entry
            # Variable to test if values entered are in range
            testRangeValues = int(str(rangeValues.get()))
            
            
            
            # Check if integer value is between 5-3000
            # Reason after grouping data for different operations and graphs
            # Type vendor seating - mean,mode,median - 61 rows entries
            # Zip codes - mean,mode,median - 1113 row entries - has been tested with 50, most relevant
            # Nr. of establishments that have committed each type of violation - 98 rows entries
            # Correlation nr. of violations committed per vendor and their zip code - 3000 rows entries
              # For correlation graph - suggested entry is 100, most relevant data. 
              # Also, it is possible for the program to crash if used with 1500 or 3000 max, as the graph displays also a legend
              # With the Facility zip codes and needs lot of memory from the computer to process the graph
            if testRangeValues < 5 or testRangeValues >=3000:
                message = f"The value you have entered is \"{testRangeValues}\".\nPlease enter a number between 5 and 3000."
                output.delete(0.0, tkinter.END)
                output.insert(tkinter.END, message)
            else:
                rangeValuesInteger = testRangeValues
                message = f"The values for generating the ouput and visualizations has been set to {rangeValuesInteger}"
                output.delete(0.0, tkinter.END)
                output.insert(tkinter.END, message)
                
            
        except ValueError:
            message = f"The value you have entered is \"{str(rangeValues.get())}\".\nPlease enter integer numbers."
            output.delete(0.0, tkinter.END)
            output.insert(tkinter.END, message)
    # Method to extract from PE Description column the seats and create a different column
    # Retain all other information in the PE Description column
    def peDescription():
        global df
        
           
        
        try:
            
            # Verify that col PE Description is present in json file
            if "PE DESCRIPTION" not in df.columns:
                              message = "Column \'PE Description \' is not present in the loaded file."
                              output.delete(0.0, tkinter.END)
                              output.insert(tkinter.END, message)
            else:
                # Check that the loaded file was not operated on
                # And the column type vendor seating does not exist
                if "TYPE VENDOR SEATING" in df.columns:
                    message = "Column \'TYPE VENDOR SEATING \' already present in the file.\n Nothing to do."
                    output.delete(0.0, tkinter.END)
                    output.insert(tkinter.END, message)
                  
                # If type vendor seating not present
                # Perform the client's requested seats operation
                else:
                     
                    # Extract with regex the seats and assign the series to a new column
                    df['TYPE VENDOR SEATING'] = df['PE DESCRIPTION'].str.extract(r'(\([^)]+\))')
                    # Replace the extracted seats with empty space
                    # Retain all other information
                    df['PE DESCRIPTION'] = df['PE DESCRIPTION'].str.replace(r'(\([^)]+\))', '', regex=True)  
                    # Feedback to user if operation was successful
                    if 'TYPE VENDOR SEATING' in df.columns:
                        message = """Column Type Vendor Seating was created.
    You can save the file at this point."""
                        output.delete(0.0, tkinter.END)
                        output.insert(tkinter.END, message)
                        print(df.columns)  
        except ValueError:
            message = "Operation to extract seats failed.\n Please check the regex."
            output.delete(0.0, tkinter.END)
            output.insert(tkinter.END, message)
        # Except file is not loaded
        except:
            message = "File containing PE DESCRIPTION is not loaded.\nPlease load file."
            output.delete(0.0, tkinter.END)
            output.insert(tkinter.END, message)
    
    # Opens a new window that generates output and visualisation for
    # inspections score per year - mean, mode, median
    # for type vendor seating
    def typeVendorSeatingWindow():
        global df        
        global rangeValuesInteger
        
            
        # open the new top level window
        scoreSeatingWindow = tkinter.Toplevel()
        scoreSeatingWindow.configure(background='#084B8A')
        # Title
        scoreSeatingWindow.title("Inspections score years 2016-2019 per vendor seating type")
        scoreSeatingWindow.geometry("800x800")
        # Style for header one
        styleHeaderOneSeating = ttk.Style()
        styleHeaderOneSeating.configure('headerOneSeating.TLabel', font = ('Times New Roman', 18),
                             pady = 4, foreground = 'white', height = 40)
            
        # Header one
        headerOne = ttk.Label(scoreSeatingWindow, text = 'Inspections score per year - vendor seating',
                              style = 'headerOneSeating.TLabel', background='#084B8A')
        headerOne.grid(row = 0, column = 1, columnspan = 12)          
                          
        # Display how many entries are used for the operations
        infoRangeValuesText = f"Generating output and visualisations for {rangeValuesInteger} values."
        infoRangeValuesLabel = tkinter.Label(scoreSeatingWindow, text=infoRangeValuesText,
                                            font = ('Times New Roman', 12),
                                            pady = 4, foreground = 'white', background='#084B8A')
        infoRangeValuesLabel.grid(row = 2, column = 1)
        # Convert datetime string to Pandas dateTime object                  
        try:  
            
            # Check if index is DatetimeIndex
            if isinstance(df.index, pd.DatetimeIndex):
                pass
            else:
                df['ACTIVITY DATE'] = pd.to_datetime(df['ACTIVITY DATE'], format = '%m/%d/%Y')
                # set the new index to the date
                df.set_index('ACTIVITY DATE', inplace=True)
            
            if 'TYPE VENDOR SEATING' in df.columns:
                year_seats_group = df.groupby(['TYPE VENDOR SEATING'])
            else:
                tkinter.messagebox.showinfo('Warning message', 'The loaded file does not contain a Type Vendor Seating column.\nPlease load required file.')
            # Calculate and store the mean for Type Vendor Seating per year
            year_seats_mean = year_seats_group.resample('BY').agg({'SCORE':'mean'}).head(rangeValuesInteger)
            # Calculate and store the mean for Type Vendor Seating per year
            year_seats_median = year_seats_group.resample('BY').agg({'SCORE':'median'}).head(rangeValuesInteger)
            # Type vendor seating mode
            year_seats_group_mode = year_seats_group['SCORE'].resample('BY').agg({'SCORE':'size'}).head(rangeValuesInteger)
            # year_seats_group_mode.unstack(level=-1)
            year_seats_group_mode_unstacked_Activity_date = year_seats_group_mode.unstack(level=-1)           
            
            # Figure and plot
            fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, ncols = 1, figsize=(6,9))
            # Creating the seaborn histogram
            sns.histplot(year_seats_mean, discrete=True, ax=ax1, label='MEAN SCORE/YEARS')
        
            sns.histplot(year_seats_median, discrete=True, ax=ax2, color='#088A08', label='MEDIAN SCORE/YEARS')
            
            sns.histplot(year_seats_group_mode_unstacked_Activity_date, ax=ax3)
             
        
            canvas = FigureCanvasTkAgg(fig, master=scoreSeatingWindow)
            canvas.draw()
            canvas.get_tk_widget().grid(row = 5, column = 1)
        
            # Creating frame for containing the toolbar
            # toolbar packs internally, but window uses grid
            frameToolbar = tkinter.Frame(scoreSeatingWindow)
            frameToolbar.grid(row = 6, column = 1)
            # creating the Matplotlib toolbar 
            toolbar = NavigationToolbar2Tk(canvas, frameToolbar)
                                    
            toolbar.update() 
            # Info about the histogram mean - ax1
        
            ax1.legend()
            ax1.set_title('MEAN, MEDIAN, MODE - INSPECTION SCORE PER YEAR\nEACH TYPE OF VENDOR SEATING')
            ax1.set_xlabel('MEAN- INSPECTION SCORE PER YEARS')
            ax1.set_ylabel('TOTAL NR.TYPE SEATS')
            # Info about the histogram mean - ax2
            ax2.legend()
            ax2.set_xlabel('MEDIAN- INSPECTION SCORE PER YEARS')
            ax2.set_ylabel('TOTAL NR.TYPE SEATS')
            # Info about the histogram mode - ax3
            
            ax3.set_xlabel('SCORE TYPES')
            ax3.set_ylabel('FREQ OF SCORES')
            # Padding to the plot
            plt.tight_layout()
        except KeyError:
            tkinter.messagebox.showinfo('Warning message', 'The loaded file does not contain an ACTIVITY DATE column.\n Please load required file for this operation.\nThis window will be closed.')
            scoreSeatingWindow.destroy()
        
        # Except activity date is not present
        except TypeError:
            tkinter.messagebox.showinfo('Warning message', 'The loaded file does not contain an ACTIVITY DATE column.\n Please load required file for this operation.\nThis window will be closed.')   
            scoreSeatingWindow.destroy()
        # If file is not loaded
        except NameError:
            tkinter.messagebox.showinfo('Warning message', 'File that contains required columns is not loaded.\nPlease load file that contains columns ACTIVITY DATE and TYPE VENDOR SEATING.\n This window will be closed.')
            scoreSeatingWindow.destroy() 
        
                 
        
        
    # Opens a new window that generates output and visualisation for
    # inspections score per year - mean, mode, median
    # for zip code
    def zipCodesWindow():
        global df
        global rangeValuesInteger
        
            
        # open the new top level window
        zipCodeWindow = tkinter.Toplevel()
        zipCodeWindow.configure(background='#084B8A')
        # Title
        zipCodeWindow.title("Inspections score years 2016-2019 per facility zip")
        zipCodeWindow.geometry("800x800")
        # Style for header one
        zipCodeHeaderOne = ttk.Style()
        zipCodeHeaderOne.configure('headerOneZip.TLabel', font = ('Times New Roman', 18),
                             pady = 4, foreground = 'white', height = 40)
        
        # Header one
        headerOne = ttk.Label(zipCodeWindow, text = 'Inspections score years 2016-2019 per zip codes',
                              style = 'headerOneZip.TLabel', background='#084B8A')
        headerOne.grid(row = 0, column = 1, columnspan = 12)
        
        # Display how many entries are used for the operations
        infoRangeValuesText = f"Generating output and visualizations for {rangeValuesInteger} values."
        infoRangeValuesLabel = tkinter.Label(zipCodeWindow, text=infoRangeValuesText,
                                            font = ('Times New Roman', 12),
                                            pady = 4, foreground = 'white', background='#084B8A')
        infoRangeValuesLabel.grid(row = 2, column = 1)
        
        try:
            
            # Check if index is DatetimeIndex 
            # Skips the error
            if isinstance(df.index, pd.DatetimeIndex):
                pass
            else:
                # Convert datetime string to Pandas dateTime object
                df['ACTIVITY DATE'] = pd.to_datetime(df['ACTIVITY DATE'], format = '%m/%d/%Y')
                # set the new index to the date
                df.set_index('ACTIVITY DATE', inplace=True)
            
            # Group data by FACILITY ZIP
            # The Zip Codes column contains thousands of 0 values
            # That were added when cleaning the data.
            # The rows were not dropped, as hundreds of stores were Active
            # FACILITY ZIP has valid entries
            if 'FACILITY ZIP' in df.columns:
                year_score_zipCodes_group = df.groupby(['FACILITY ZIP'])
            else:
                tkinter.messagebox.showinfo('Warning message', 'The loaded file does not contain an Zip Codes column.\nPlease load required file.')
            # Calculate and store the mean for Zip Codes per years
            zipCodesMean = year_score_zipCodes_group.resample('BY').agg({'SCORE':'mean'}).head(rangeValuesInteger)
            # Calculate and store the mean for Zip Codes per years
            zipCodesMedian = year_score_zipCodes_group.resample('BY').agg({'SCORE':'median'}).head(rangeValuesInteger)
            # Type vendor seating mode
            zipCodesMode = year_score_zipCodes_group['SCORE'].resample('BY').agg({'SCORE':'size'}).head(rangeValuesInteger)
            # zipCodesMode.unstack(level=0)
            zipCodesMode_unstacked_Activity_date = zipCodesMode.unstack(level=-1)   
        
            # Figure and plot
            fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, ncols = 1, figsize=(6,9))
            # Creating the seaborn histogram
            sns.histplot(zipCodesMean, discrete=True, ax=ax1, label='MEAN SCORE/YEARS')
        
            sns.histplot(zipCodesMedian, discrete=True, ax=ax2, color='#088A08', label='MEDIAN SCORE/YEARS')
        
            sns.histplot(zipCodesMode_unstacked_Activity_date, ax=ax3) 
        
            canvas = FigureCanvasTkAgg(fig, master=zipCodeWindow)
            canvas.draw()
            canvas.get_tk_widget().grid(row = 5, column = 1)
        
            # Creating frame for containing the toolbar
            # toolbar packs internally, but window uses grid
            frameToolbar = tkinter.Frame(zipCodeWindow)
            frameToolbar.grid(row = 6, column = 1)
            # creating the Matplotlib toolbar 
            toolbar = NavigationToolbar2Tk(canvas, frameToolbar)
                                    
            toolbar.update() 
            # Info about the histogram mean - ax1
        
            ax1.legend()
            ax1.set_title('MEAN, MEDIAN, MODE - INSPECTION SCORE PER YEAR\nCALCULATED PER FACILITY ZIP')
            ax1.set_xlabel('SCORE MEAN VALUES 2016-2019')
            ax1.set_ylabel('TOTAL NR.TYPE SEATS')
            # Info about the histogram mean - ax2
            ax2.legend()
            ax2.set_xlabel('SCORE MEDIAN VALUES 2016-2019')
            ax2.set_ylabel('TOTAL NR.TYPE SEATS')
            # Info about the histogram mode - ax3
        
            ax3.set_xlabel('SCORE TYPES FREQUENCY 2016-2019')
            ax3.set_ylabel('TOTAL NR OF SCORE TYPES')
            # Padding to the plot
            plt.tight_layout()
            
        except KeyError:
            tkinter.messagebox.showinfo('Warning message', 'The loaded file does not contain an ACTIVITY DATE column.\n Please load required file for this operation.\nThis window will be closed.')
            zipCodeWindow.destroy()
        # File needs to contain Activity Date column in order to
        # Set the index as DateTimeIndex
        except TypeError:
            tkinter.messagebox.showinfo('Warning message', 'The loaded file does not contain an ACTIVITY DATE column.\n Please load required file for this operation.\nThis window will be closed.')
            zipCodeWindow.destroy()        
            
        # Catching df not defined error
        # If file is not loaded
        except NameError:
            tkinter.messagebox.showinfo('Warning message', 'File that contains required columns is not loaded.\nPlease load file. This window will be closed.')
            zipCodeWindow.destroy() 
        
            
    # New window that displays in a graph the number 
    # of establishments that have committed each type of violation
    # Required loading inspection and violation files
    def graphViolationWindow():
        global df
        global dfViolations
        global rangeValuesInteger
        

            
        # open the new top level window    
        graphViolationWindow = tkinter.Toplevel()
        graphViolationWindow.configure(background='#084B8A')
        # Title
        graphViolationWindow.title("Establishments and violations committed")
        graphViolationWindow.geometry("800x800")
        # Style for header one
        graphViolationHeaderOne = ttk.Style()
        graphViolationHeaderOne.configure('headerOneViolations.TLabel', font = ('Times New Roman', 18),
                             pady = 4, foreground = 'white', height = 40)
        
        # Header one
        headerOne = ttk.Label(graphViolationWindow, text = 'Number of establishments that have committed each type of violation.\nTime period 2016-2019',
                              style = 'headerOneViolations.TLabel', background='#084B8A')
        headerOne.grid(row = 0, column = 1, columnspan = 12)
        
        # Display how many entries are used for the operations
        infoRangeValuesText = f"Generating output and visualizations for {rangeValuesInteger} values."
        infoRangeValuesLabel = tkinter.Label(graphViolationWindow, text=infoRangeValuesText,
                                            font = ('Times New Roman', 12),
                                            pady = 4, foreground = 'white', background='#084B8A')
        infoRangeValuesLabel.grid(row = 2, column = 1)
        
        try:
            # Merge the two global datasets on col SERIAL NUMBER
            dfMerged = dfViolations.merge(df, on='SERIAL NUMBER', how='outer')
            # Group by violation code column
            violations_code_group = dfMerged.groupby('VIOLATION CODE')
            total_values_violations_code_facility_name = violations_code_group['FACILITY NAME'].size()
        
            # Y axis
            yAxis = total_values_violations_code_facility_name.sort_values(ascending=False).head(rangeValuesInteger).index
            # X axis
            xAxis = total_values_violations_code_facility_name.sort_values(ascending=False).head(rangeValuesInteger).values
            # Create new violations dataFrame
            dfViolationsEstablishments = pd.DataFrame(xAxis, yAxis)
        
            # Figure and plot
            fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(6,9))
            # Creating the seaborn barChart
            sns.barplot(x=dfViolationsEstablishments[0], y=dfViolationsEstablishments.index, data=dfViolationsEstablishments, ax=ax, orient='h')
         
        
            canvas = FigureCanvasTkAgg(fig, master=graphViolationWindow)
            canvas.draw()
            canvas.get_tk_widget().grid(row = 4, column = 1)
        
            # Creating frame for containing the toolbar
            # toolbar packs internally, but window uses grid
            frameToolbar = tkinter.Frame(graphViolationWindow)
            frameToolbar.grid(row = 5, column = 1)
            # creating the Matplotlib toolbar 
            toolbar = NavigationToolbar2Tk(canvas, frameToolbar)
                                     
            # Info about the histogram mean - ax1
            ax.set_xlabel('Nr. of establishments - 2016-2019')
            ax.set_ylabel('Type of violation')
            plt.tight_layout()
        # Catching not defined error
        # If either global df or dfViolations file is not loaded
        # with cleaned data
        except NameError:
            tkinter.messagebox.showinfo('Warning message', 'The requested operation requires two files: the cleaned inspection and violations files.\nPlease load first inspections file. Then, load the violations file.\n This window will be closed.')
            graphViolationWindow.destroy()
        # Catching the KeyError on 'SERIAL NUMBER'
        # If either first or second file does not contain the required 'SERIAL NUMBER' column
        # Warn the user and close window
        except KeyError:
            tkinter.messagebox.showinfo('Warning message', 'Could not perform merge operation on the two loaded files.\nMake sure the two files are the cleaned inspections and violations and both contain SERIAL NUMBER column.\nThis window will be closed.')
            graphViolationWindow.destroy()
    
    
    # New window that displays a scatter plot with
    # the correlation between number of violations per vendor
    # and their zip code
    def correlationViolationWindow():
        global df
        global dfViolations
        
        
        # open the new top level window
        correlationViolationWindow = tkinter.Toplevel()
        correlationViolationWindow.configure(background='#084B8A')
        # Title
        correlationViolationWindow.title("Correlation between the number of violations committed per vendor and their zip code")
        correlationViolationWindow.geometry("800x800")
        # Style for header one
        correlationViolationHeaderOne = ttk.Style()
        correlationViolationHeaderOne.configure('headerOneViolations.TLabel', font = ('Times New Roman', 18),
                             pady = 4, foreground = 'white', height = 40)
        
        # Header one
        headerOne = ttk.Label(correlationViolationWindow, text = 'Correlation between violations committed per vendor\n and their zip code (years:2016-2019)',
                              style = 'headerOneViolations.TLabel', background='#084B8A')
        headerOne.grid(row = 0, column = 1, columnspan = 12)
        try:
            # Merge the two global datasets on col SERIAL NUMBER
            dfMerged = dfViolations.merge(df, on='SERIAL NUMBER', how='outer')
    
            # Group by facility id column
            dfMerged_facility_id_group = dfMerged.groupby(['FACILITY ID'])
            # Calculate total number of violations per vendor
            # Only first 100 entries out of 3000
            # For rendering in a fast manner the graph and its legend
            # I have selected only first 100 entries.
            total_nr_violation_code_per_vendor_id = dfMerged_facility_id_group['VIOLATION CODE'].size().sort_values(ascending=False).head(100)
        
            # Group by facility zip
            dfMerged_facility_zip_group = dfMerged.groupby('FACILITY ZIP')
            # # Calculate total number of violations per facility zip
            total_nr_violation_code_per_facility_zip = dfMerged_facility_zip_group['VIOLATION CODE'].size().sort_values(ascending=False).head(100)
        
            # Create new DataFrame with total violation codes per vendor and their zip code
            frame = {'FACILITY ID':total_nr_violation_code_per_vendor_id.index, 'TOTAL VIOLATIONS/FACILITY ID':total_nr_violation_code_per_vendor_id.values, 'FACILITY ZIP':total_nr_violation_code_per_facility_zip.index,'TOTAL VIOLATIONS/FACILITY ZIP':total_nr_violation_code_per_facility_zip.values}
            dfViolationsZipCodeFacilityZip = pd.DataFrame(frame)
        
            # Figure and plot
            fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(10,9))
            # Creating the seaborn scatterplot
            sns.scatterplot(x = total_nr_violation_code_per_vendor_id.values, y = total_nr_violation_code_per_facility_zip.values, hue='FACILITY ID', linewidth=1, data=dfViolationsZipCodeFacilityZip, ax=ax)
         
            # Canvas to hold the seaborn graph
            canvas = FigureCanvasTkAgg(fig, master=correlationViolationWindow)
            canvas.draw()
            canvas.get_tk_widget().grid(row = 2, column = 1)
        
            # Creating frame for containing the toolbar
            # toolbar packs internally, but window uses grid
            frameToolbar = tkinter.Frame(correlationViolationWindow)
            frameToolbar.grid(row = 4, column = 1)
            # creating the Matplotlib toolbar 
            toolbar = NavigationToolbar2Tk(canvas, frameToolbar)
                                    
            #toolbar.update() 
            # Info about the histogram mean - ax1
            ax.set_xlabel('TOTAL NR. OF VIOLATIONS PER FACILITY ID')
            ax.set_ylabel('TOTAL NR. OF VIOLATIONS PER FACILITY ZIP')                                                                                                                            
            ax.set_yscale('log')
        # Catching not defined error
        # If either global df or dfViolations file is not loaded
        # with cleaned data
        except NameError:
            tkinter.messagebox.showinfo('Warning message', 'The requested operation requires two files: the cleaned inspection and violations files.\nPlease load first inspections file. Then, load the violations file.\n This window will be closed.')
            correlationViolationWindow.destroy()
        # Catching the KeyError on 'SERIAL NUMBER'
        # If either first or second file does not contain the required 'SERIAL NUMBER' column
        # Warn the user and close window
        except KeyError:
            tkinter.messagebox.showinfo('Warning message', 'Could not perform merge operation on the two loaded files.\nMake sure the two files are the cleaned inspections and violations and both contain SERIAL NUMBER column.\nThis window will be closed.')
            correlationViolationWindow.destroy()
        
        
    
    
    
    # Style zone for interface elements
    
    # Style for header one
    styleHeaderOne = ttk.Style()
    styleHeaderOne.configure('headerOne.TLabel', font = ('Times New Roman', 20),
                             pady = 4, background='#084B8A', foreground = 'white', height = 40)
    
    # Style for Frame - containerInteractions
    styleContainer = ttk.Style()
    styleContainer.configure('container.TFrame', width = 60, height = 400)
    
        
    # Interface 
    
    # Menu
    menuBar = tkinter.Menu(rootWindow)
    fileMenu = tkinter.Menu(menuBar, tearoff = 0, font = ('Times New Roman', 16), foreground = 'black')
    fileMenu.add_command(label='Load and clean csv', command = openInitialData)
    fileMenu.add_command(label='Load prepared data', command= loadCleanedData)
    fileMenu.add_command(label='Save prepared data', command = saveCleanedData)
    fileMenu.add_command(label='Quit', command= on_closing)
    menuBar.add_cascade(label='File', menu=fileMenu, font = ('Times New Roman', 16),
                        background = '#084B8A', foreground = 'black')
    
    
    # Header one
    headerOne = ttk.Label(rootWindow, text = 'STORE AUDIT', style = 'headerOne.TLabel')
    headerOne.place(x = 330, y = 20)
    
    
    # Container Frame for holding interactive buttons
    containerInteractions = ttk.Frame(rootWindow, style = 'container.TFrame')
    containerInteractions.place( x = 20, y = 200)
    
    # PE Description button
    peButton = tkinter.Button(containerInteractions, activebackground = '#045FB4',
                              text = 'PE Description\nExtract nr. and type of seat', bg = '#0080FF',
                              fg = 'white', width = 20, padx = 10, 
                              font = ('Times New Roman', 14), command = peDescription )
    peButton.grid(row = 1, column = 0)
    
    # Manipulating the range of values to be used
    # for output and visualization(s)

    inputRangeValuesLabel = tkinter.Label(containerInteractions, text='Enter numerical values(5-3000).\nWill be used for output and graphs.', 
                                         bg = '#0080FF', activebackground = '#045FB4',fg = 'white', 
                             width = 31, padx = 2, font = ('Times New Roman', 10))
    inputRangeValuesEntry = tkinter.Entry(containerInteractions, textvariable=rangeValues,
                                         width = 37,  font = ('Times New Roman', 10))
    
    # Button that calls a method to retrieve the entry
    inputRangeValuesButton = tkinter.Button(containerInteractions, text = 'Apply value',
                             bg = '#0080FF', activebackground = '#045FB4',fg = 'white',
                             width = 20, padx = 10, font = ('Times New Roman', 14), command = getEntryValue)
    inputRangeValuesLabel.grid(row = 2, column = 0)
    inputRangeValuesEntry.grid(row = 3, column = 0)
    inputRangeValuesButton.grid(row = 4, column = 0)
    
    # Inspection score per year vendor seating button
    insp_score_per_year_vendor_seating = tkinter.Button(containerInteractions, text = 'Insp. score per year\n for type of vendor seating',
                             bg = '#0080FF', activebackground = '#045FB4',fg = 'white', 
                             width = 20, padx = 10, font = ('Times New Roman', 14),command = typeVendorSeatingWindow)
    insp_score_per_year_vendor_seating.grid(row = 5, column = 0)
    
    # Inspection score per year zip code button
    insp_score_per_year_zip_codes = tkinter.Button(containerInteractions, text = 'Insp. score per year\n for each facility zip code',
                             bg = '#0080FF', activebackground = '#045FB4',fg = 'white',
                             width = 20, padx = 10, font = ('Times New Roman', 14), command = zipCodesWindow)
    insp_score_per_year_zip_codes.grid(row = 6, column = 0)
    
    # Graph that displays the number of establishments that have committed each type of violation
    graphViolationButton = tkinter.Button(containerInteractions, text = 'Graph nr. of establishments\n that have committed\neach type of violation',
                             bg = '#0080FF', activebackground = '#045FB4',fg = 'white', width = 20,
                            padx = 10, font = ('Times New Roman', 14), command = graphViolationWindow)
    graphViolationButton.grid(row = 7, column = 0)
    
    # Correlation between number of violations committed per vendor and zip code
    zipCodeViolationVendor = tkinter.Button(containerInteractions, text = 'Correlation nr. of violations\n committed per vendor\nand their zip code ',
                             bg = '#0080FF', activebackground = '#045FB4',
                            fg = 'white', width = 20, padx = 10, font = ('Times New Roman', 14), command = correlationViolationWindow)
    zipCodeViolationVendor.grid(row = 8, column = 0)
    
    
    
    
    
    # To display outputs - useful messages for user
    
    # Container for output and label
    containerOutput = tkinter.Frame(rootWindow, bg = "#0080FF")
    containerOutput.place( x = 300, y = 200)
    # Label for output
    outputLabel = tkinter.Label(containerOutput, text = "Output message area", font = ('Times New Roman', 16),
                                bg = '#0080FF', activebackground = '#045FB4',fg = 'white', )
    
    # Output box
    output = tkinter.Text(containerOutput, width=36, height=12,
                     wrap='word', background='#0080FF', pady=5, foreground = 'white')
    outputLabel.grid( row = 0, column = 6)
    output.grid( row = 2, column = 6)
    
    # Display the menu bar
    rootWindow.configure(menu = menuBar, background = '#084B8A')
    # Callback on_closing and display message box
    rootWindow.protocol("WM_DELETE_WINDOW", on_closing)
    # mainloop
    tkinter.mainloop()

In [18]:
if __name__ == "__main__":
    
    main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\samue\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-17-05bd0b54af1e>", line 144, in saveCleanedData
    df.to_json(saveFileLocation)
NameError: name 'df' is not defined


In [19]:
print(df.columns)

NameError: name 'df' is not defined